## Deliverable 2. Create a Customer Travel Destinations Map.

In [14]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [15]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Ushuaia,-54.8000,-68.3000,59.00,63,75,19.57,AR,moderate rain
1,1,Eisenerz,47.5333,14.8833,44.60,65,75,6.91,AT,broken clouds
2,2,Lawrenceburg,38.0373,-84.8966,71.60,56,1,11.50,US,clear sky
3,3,Cabedelo,-6.9811,-34.8339,82.40,78,20,9.22,BR,few clouds
4,4,Upernavik,72.7868,-56.1549,-25.28,88,18,4.94,GL,few clouds


In [16]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 80


In [17]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
selected_cities = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
selected_cities.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
2,2,Lawrenceburg,38.0373,-84.8966,71.60,56,1,11.50,US,clear sky
6,6,Katghora,22.5000,82.5500,71.44,18,0,6.24,IN,clear sky
9,9,Bethel,41.3712,-73.4140,64.40,36,20,8.05,US,few clouds
10,10,Tessalit,20.1986,1.0114,73.15,8,0,14.14,ML,clear sky
12,12,Kutum,14.2000,24.6667,68.29,17,4,17.60,SD,clear sky
13,13,Camana,-16.6228,-72.7111,73.74,71,49,5.84,PE,scattered clouds
16,16,Bredasdorp,-34.5322,20.0403,60.80,67,36,4.61,ZA,scattered clouds
17,17,Portland,45.5234,-122.6762,60.01,54,40,1.61,US,scattered clouds
23,23,Lompoc,34.6391,-120.4579,73.00,40,1,13.80,US,clear sky
29,29,Kalmunai,7.4167,81.8167,78.03,81,79,1.88,LK,broken clouds


In [18]:
# 4a. Determine if there are any empty rows.
selected_cities.count()

City_ID                277
City                   277
Lat                    277
Lng                    277
Max Temp               277
Humidity               277
Cloudiness             277
Wind Speed             277
Country                273
Current Description    277
dtype: int64

In [19]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities = selected_cities.dropna()

In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Lawrenceburg,US,71.60,clear sky,38.0373,-84.8966,
6,Katghora,IN,71.44,clear sky,22.5000,82.5500,
9,Bethel,US,64.40,few clouds,41.3712,-73.4140,
10,Tessalit,ML,73.15,clear sky,20.1986,1.0114,
12,Kutum,SD,68.29,clear sky,14.2000,24.6667,
13,Camana,PE,73.74,scattered clouds,-16.6228,-72.7111,
16,Bredasdorp,ZA,60.80,scattered clouds,-34.5322,20.0403,
17,Portland,US,60.01,scattered clouds,45.5234,-122.6762,
23,Lompoc,US,73.00,clear sky,34.6391,-120.4579,
29,Kalmunai,LK,78.03,broken clouds,7.4167,81.8167,


In [23]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [43]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Lawrenceburg,US,71.60,clear sky,38.0373,-84.8966,Best Western Lawrenceburg Inn
6,Katghora,IN,71.44,clear sky,22.5000,82.5500,Deppak bhojanalay new bus stand
9,Bethel,US,64.40,few clouds,41.3712,-73.4140,Courtyard by Marriott Danbury
12,Kutum,SD,68.29,clear sky,14.2000,24.6667,Dibbo areaمنطقة دبو
13,Camana,PE,73.74,scattered clouds,-16.6228,-72.7111,Hostal Morales
16,Bredasdorp,ZA,60.80,scattered clouds,-34.5322,20.0403,Bredasdorp Country Manor
17,Portland,US,60.01,scattered clouds,45.5234,-122.6762,Embassy Suites by Hilton Portland Downtown
23,Lompoc,US,73.00,clear sky,34.6391,-120.4579,Holiday Inn Express Lompoc
29,Kalmunai,LK,78.03,broken clouds,7.4167,81.8167,VS Villa
30,Port Elizabeth,ZA,62.60,few clouds,-33.9180,25.5701,39 On Nile Guest House


In [45]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [52]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [53]:
# 11a. Add a marker layer for each city to the map. 

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.


fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)


fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))